In [1]:
# 01_limpieza_columnas.ipynb

import pandas as pd

# Cargar el archivo CSV tratado
df = pd.read_csv('data/telecomx_data.csv')

# Ver las primeras filas
df.head()


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [9]:
print(df.columns)


Index(['Churn', 'customer', 'phone', 'internet', 'account'], dtype='object')


In [10]:
df.to_csv('data/telecomx_data_limpio.csv', index=False)


In [11]:
df_encoded = pd.get_dummies(df, drop_first=True)


In [12]:
df_encoded.to_csv('data/telecomx_data_encoded.csv', index=False)


In [13]:
# Ver distribución de la variable Churn
print(df['Churn'].value_counts())

# Ver proporciones relativas
print(df['Churn'].value_counts(normalize=True))


Churn
No     5174
Yes    1869
Name: count, dtype: int64
Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64


In [20]:
print(df.dtypes)


Churn       object
customer    object
phone       object
internet    object
account     object
dtype: object


In [21]:
df_encoded = pd.get_dummies(df, drop_first=True)


In [22]:
correlaciones = df_encoded.corr()


In [4]:
import pandas as pd

df = pd.read_csv('data/telecomx_data.csv')


In [5]:
df_reducido = df.drop(['customer', 'phone', 'account'], axis=1)


In [6]:
df_encoded = pd.get_dummies(df_reducido, drop_first=True)


In [7]:
correlaciones = df_encoded.corr()


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:

import os, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# 0) Ubicación y verificación
print("Working dir:", os.getcwd())
os.makedirs("data", exist_ok=True)
os.makedirs("figures", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

# 1) Localizar automáticamente el CSV en ./data
csvs = glob.glob("data/*.csv")
if not csvs:
    raise FileNotFoundError("No encontré archivos .csv en la carpeta 'data/'. "
                            "Mete tu dataset en esa carpeta o indica la ruta exacta.")
csv_path = csvs[0]  # si hay varios, elige el correcto manualmente
print("Usando archivo:", csv_path)

# 2) Cargar dataset original
df = pd.read_csv(csv_path)

# 3) Crear df_reducido 

cols_drop = []  # p.ej.: ["customerID", "unnamed", "notas"]
df_reducido = df.drop(columns=[c for c in cols_drop if c in df.columns], errors="ignore")

# 4) Definir nombre de la variable objetivo (AJUSTA AL TUYO)
target_col = "Churn"  # ej: "Churn", "Canceled", "churn"
if target_col not in df_reducido.columns:
    raise KeyError(f"No encontré la columna objetivo '{target_col}' en el dataset. "
                   f"Columnas disponibles: {list(df_reducido.columns)[:15]}...")

# Si el target es texto Yes/No, mapeamos
if df_reducido[target_col].dtype == "O":
    df_reducido[target_col] = df_reducido[target_col].str.strip().str.title().map({"Yes":1, "No":0}).fillna(df_reducido[target_col])

# 5) One-hot encoding (todo numérico)
df_encoded = pd.get_dummies(df_reducido, drop_first=True, dtype=int)

# Limpiar infinitos/nulos
X = df_encoded.replace([np.inf, -np.inf], np.nan).fillna(0)

print("Shape df_encoded:", X.shape)

# 6) Matriz de correlación y correlación con el target
correlaciones = X.corr(numeric_only=True)
if target_col not in correlaciones.columns:
    raise KeyError(f"Después del encoding, no aparece '{target_col}' en la matriz. "
                   "Revisa que no lo hayas eliminado o renombrado.")

corr_target = correlaciones[target_col].sort_values(ascending=False)

# 7) Guardar salidas
correlaciones.to_csv("outputs/corr_matrix.csv")
corr_target.to_csv("outputs/corr_with_target.csv")
print("✓ Guardado outputs/corr_matrix.csv y outputs/corr_with_target.csv")

# 8) Gráficos
plt.figure(figsize=(12,10))
plt.imshow(correlaciones, aspect='auto')
plt.title("Matriz de correlación (Pearson)")
plt.colorbar()
plt.tight_layout()
plt.savefig("figures/corr_heatmap.png", dpi=150)
plt.close()

topN = 15
top_feats = corr_target.abs().sort_values(ascending=False).head(topN).index
sub = correlaciones.loc[top_feats, top_feats]
plt.figure(figsize=(8,6))
plt.imshow(sub, aspect='auto')
plt.xticks(range(len(top_feats)), top_feats, rotation=90)
plt.yticks(range(len(top_feats)), top_feats)
plt.title(f"Top {topN} correlaciones (abs) con {target_col}")
plt.colorbar()
plt.tight_layout()
plt.savefig("figures/corr_topN_heatmap.png", dpi=150)
plt.close()
print("✓ Guardado figures/corr_heatmap.png y figures/corr_topN_heatmap.png")

# 9) Vista rápida de resultados
print("\nTop 20 variables por |r| con el target:")
print(corr_target.abs().sort_values(ascending=False).head(20))


Working dir: /Users/Aracely/Desktop/telecomx-churn-analysis-parte2
Usando archivo: data/telecomx_data.csv


/var/folders/7q/zrybffg57h31156bt30slz600000gp/T/ipykernel_48817/4217840600.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_reducido[target_col] = df_reducido[target_col].str.strip().str.title().map({"Yes":1, "No":0}).fillna(df_reducido[target_col])


Shape df_encoded: (7267, 15217)


In [4]:

import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Cargar dataset ---
csv_path = "data/telecomx_data.csv"   # tu archivo
df = pd.read_csv(csv_path)

# --- Definir target ---
target_col = "Churn"  

# Estandarizar target 
y = df[target_col]
if y.dtype == "O":
    y = y.astype(str).str.strip().str.title().map({"Yes": 1, "No": 0})
y = y.fillna(0).astype(int)



drop_base = ["customerID"]  
df_reducido = df.drop(columns=[c for c in drop_base if c in df.columns], errors="ignore")

# --- Detectar categóricas de alta cardinalidad y filtrarlas ---
cat_cols = [c for c in df_reducido.columns if df_reducido[c].dtype == "O" and c != target_col]
uniques = df_reducido[cat_cols].nunique().sort_values(ascending=False)
THRESH = 30  # umbral de cardinalidad
high_card = uniques[uniques > THRESH].index.tolist()


df_tmp = df_reducido.drop(columns=high_card, errors="ignore")


if target_col in df_tmp.columns:
    df_tmp = df_tmp.drop(columns=[target_col])

X = pd.get_dummies(df_tmp, drop_first=True, dtype=int)
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

print("Shapes -> X:", X.shape, "| y:", y.shape)


corr_target = X.corrwith(y).sort_values(key=lambda s: s.abs(), ascending=False)

# --- Guardar salidas ---
os.makedirs("outputs", exist_ok=True)
os.makedirs("figures", exist_ok=True)
corr_target.to_csv("outputs/corr_with_target.csv", index=True)
print("✓ outputs/corr_with_target.csv guardado")

# --- Heatmap pequeño con Top N ---
topN = 15
top_feats = corr_target.abs().head(topN).index.tolist()
sub = pd.concat([X[top_feats], y.rename(target_col)], axis=1).corr()

plt.figure(figsize=(8,6))
plt.imshow(sub, aspect='auto')
plt.xticks(range(len(sub.columns)), sub.columns, rotation=90)
plt.yticks(range(len(sub.index)), sub.index)
plt.title(f"Top {topN} correlaciones (abs) con {target_col}")
plt.colorbar()
plt.tight_layout()
plt.savefig("figures/corr_topN_heatmap.png", dpi=150)
plt.close()
print("✓ figures/corr_topN_heatmap.png guardado")

# --- Vista rápida ---
print("\nTop 20 |r| con el target:")
print(corr_target.abs().head(20))


Shapes -> X: (7267, 2) | y: (7267,)
✓ outputs/corr_with_target.csv guardado
✓ figures/corr_topN_heatmap.png guardado

Top 20 |r| con el target:
phone_{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}    0.039343
phone_{'PhoneService': 'Yes', 'MultipleLines': 'No'}     0.031430
dtype: float64


In [3]:
import pandas as pd

# Cargar el dataset original (ajusta el nombre al tuyo dentro de /data)
df = pd.read_csv("data/telecomx_data.csv")  # cámbialo si el archivo se llama distinto

# Mira todas las columnas
print(df.columns.tolist())

# Buscar columnas que contengan "churn" o "cancel"
[col for col in df.columns if "churn" in col.lower() or "cancel" in col.lower()]



['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']


['Churn']